<a href="https://colab.research.google.com/github/11jolek11/BigData/blob/yfinance/List5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets list

In [ ]:
!kaggle datasets download -d shivam2503/diamonds
!kaggle datasets download -d datascientistanna/customers-dataset
!mkdir dataset
!unzip -o -q /content/diamonds.zip -d /content/dataset
!unzip -o -q /content/customers-dataset.zip -d /content/dataset

In [ ]:
import seaborn as sns
from sklearn import datasets
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import TruncatedSVD

In [ ]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

# Load data

In [ ]:
iris = datasets.load_iris(as_frame=True)
wine = datasets.load_wine(as_frame=True)
california_housing = datasets.fetch_california_housing(as_frame=True)
diamonds_df = pd.read_csv("/content/dataset/diamonds.csv", sep=",", header=0)
customers_df = pd.read_csv("/content/dataset/Customers.csv", sep=",", header=0)

In [ ]:
diamonds_df.drop(['Unnamed: 0'], axis=1, inplace=True)

Iris

In [ ]:
iris.target_names

In [ ]:
iris.frame

In [ ]:
iris.frame.info()

In [ ]:
sns.heatmap(iris.frame.corr(), annot=True, cmap='rainbow')

In [ ]:
sns.pairplot(iris.frame, hue='target', height=2.5)

Wine

In [ ]:
wine.frame

In [ ]:
wine.target

In [ ]:
wine.frame.info()

In [ ]:
fig, axes = plt.subplots(figsize=(10,10))
sns.heatmap(wine.frame.corr(), annot=True, cmap='crest', robust=True, ax=axes)

In [ ]:
sns.pairplot(wine.frame, hue='target', height=2.5)

California

In [ ]:
california_housing.frame.info()

In [ ]:
fig, axes = plt.subplots(figsize=(10,10))
sns.heatmap(california_housing.frame.corr(), annot=True, cmap='crest', robust=True, ax=axes)

In [ ]:
temp_scaler = StandardScaler(copy=True)

standardized_housing = pd.DataFrame(temp_scaler.fit_transform(california_housing.frame), columns=california_housing.frame.columns)
sns.jointplot(data=standardized_housing, x='MedHouseVal', y='MedInc', kind="kde")

Diamonds

In [ ]:
diamonds_df.info()

In [ ]:
diamonds_df.head()
# TODO: how analyze nonnumerical data?

In [ ]:
fig, axes = plt.subplots(figsize=(10,10))
sns.heatmap(diamonds_df.corr(), annot=True, cmap='crest', robust=True, ax=axes)

In [ ]:
# standardized_diamonds = pd.DataFrame(temp_scaler.fit_transform(diamonds_df), \
# columns=diamonds_df.columns)
diamonds_df.head()
numerical = ['carat', 'depth', 'table','price', 'x', 'y', 'z']
categorical = ['cut', 'color', 'clarity']

diamonds_color = diamonds_df['color']

In [ ]:
diamonds_df.drop(categorical, inplace=True, axis=1)

Customers


In [ ]:
customers_df

In [ ]:
encoder = OrdinalEncoder()

customers_df['Gender'] = encoder.fit_transform(customers_df.Gender.values.reshape(-1, 1))
customers_df['Profession'] = encoder.fit_transform(customers_df.Profession.values.reshape(-1, 1))

In [ ]:
fig, axes = plt.subplots(figsize=(10,10))
sns.heatmap(customers_df.corr(), annot=True, cmap='crest', robust=True, ax=axes)

In [ ]:
sns.pairplot(customers_df, hue='Profession', height=2.5)

In [ ]:
sns.pairplot(customers_df, hue='Gender', height=2.5)

# Zad 1

In [ ]:
pca_pipeline = Pipeline(
    [
      ('scaler', StandardScaler()),
      ('pca_dim_reduction', PCA(n_components=2)),
    ]
)

In [ ]:
iris_target = iris.frame['target']
iris_df = iris.frame.drop(['target'], axis=1)

In [ ]:
iris_pca = pd.DataFrame(data=pca_pipeline.fit_transform(iris_df), columns = ['pca1', 'pca2'])

In [ ]:
sns.scatterplot(data=iris_pca, x='pca1', y='pca2', hue=iris.frame['target'])

In [ ]:
wine_df = wine.frame.drop(['target'], axis=1)

In [ ]:
wine_pca = pd.DataFrame(data = pca_pipeline.fit_transform(wine_df), columns = ['pca1', 'pca2'])

In [ ]:
sns.scatterplot(data=wine_pca, x='pca1', y='pca2', hue=wine.frame['target'])

In [ ]:
customers_df = customers_df.dropna().reset_index(drop=True)
customers_temp = customers_df.drop(['Gender', 'Profession'], axis=1)
customers_temp.head()
# print(customers_df.shape)

In [ ]:
customers_pca = pd.DataFrame(pca_pipeline.fit_transform(customers_temp), columns = ['pca1', 'pca2'])
sns.scatterplot(data=customers_pca, x='pca1', y='pca2', hue=customers_df['Gender'])
# TODO: finde better dataset

# Zad 2

In [ ]:
# california_housing.frame.info()
california_pca = pd.DataFrame(pca_pipeline.fit_transform(california_housing.frame), columns = ['pca1', 'pca2'])

In [ ]:
sns.scatterplot(data=california_pca, x='pca1', y='pca2')

In [ ]:
diamonds_df.head()

In [ ]:
diamonds_pca = pd.DataFrame(pca_pipeline.fit_transform(diamonds_df), columns = ['pca1', 'pca2'])

In [ ]:
sns.scatterplot(data=diamonds_pca, x='pca1', y='pca2')

# Zad 3

In [ ]:
standarization_pipeline = Pipeline(
    [
      ('scaler', StandardScaler()),
    ]
)

lda = LDA(n_components=2)

In [ ]:
iris_stand = pd.DataFrame(data=standarization_pipeline.fit_transform(iris_df))
iris_lda = lda.fit_transform(iris_stand, iris_target)

In [ ]:
wine_df
wine_stand = pd.DataFrame(data=standarization_pipeline.fit_transform(wine_df))

wine_lda = lda.fit_transform(wine_stand, wine.target)

In [ ]:
# customers_lda = pd.DataFrame(pca_pipeline.fit_transform(customers_temp), columns = ['lda1', 'lda2'])
# sns.scatterplot(data=customers_lda, x='lda1', y='lda2')
customers_profession = customers_df['Profession']
# customers_df.drop(['Profession'], inplace=True, axis=1)

customers_stand = pd.DataFrame(data=standarization_pipeline.fit_transform(customers_df))
customers_lda = lda.fit_transform(customers_stand, customers_profession)

In [ ]:
# california_lda = pd.DataFrame(pca_pipeline.fit_transform(california_housing.frame), columns = ['lda1', 'lda2'])
# sns.scatterplot(data=wine_lda, x='lda1', y='lda2')
california_stand = pd.DataFrame(data=standarization_pipeline.fit_transform(california_housing.frame))
california_lda = lda.fit_transform(california_stand, california_housing.frame.HouseAge)
# california_housing.frame

In [ ]:
diamonds_stand = pd.DataFrame(data=standarization_pipeline.fit_transform(diamonds_df))
diamonds_lda = lda.fit_transform(diamonds_stand, diamonds_color)
# diamonds_lda = pd.DataFrame(pca_pipeline.fit_transform(diamonds_df), columns = ['lda1', 'lda2'])
# sns.scatterplot(data=wine_lda, x='lda1', y='lda2')

# Zad 4

In [ ]:
# svd_pipeline = Pipeline(
#     [
#       ('scaler', StandardScaler()),
#       ('svd_dim_reduction', TruncatedSVD()),
#     ]
# )

svd = TruncatedSVD(n_components=2)

In [ ]:
# iris_stand
svd.fit_transform(iris_stand)

In [ ]:
svd.fit_transform(california_stand)

In [ ]:
svd.fit_transform(wine_stand)

In [ ]:
svd.fit_transform(customers_stand)

In [ ]:
svd.fit_transform(diamonds_stand)